# Extract analysis results and perfrom verifications

This script reads out specified values form the result dict of each structure of a sampling iteration and saves the results in a df.

From the read out strains and stresses the minimum structural safety $\eta$ ("Erfüllungsgrad") are calculated for each structure.

Then the calculated verification results and the corresponding bridge parameter are saved in a csv-file..

In [1]:
#import packages
import pandas as pd
import os

## Batch Evaluation

In [7]:
# Input HERE
idx_s=318
start_id = 0
end_id = 14
step='step_4'
standard_directory=True #Please specify False if your data does not lie in the standard directory in the repository and you would like to specify the path
if not standard_directory: # please specify the path of standard_directory is set to False (if set to True the following path and folder_name is ignored)
    prepath=None#"T:\\02_Forschung\\2022_SBB_LoA\\03_Phase_2\\04_WP_2.2\\07_DataGeneration\\240718_MS3_10000_LHS_CFB_withOldGH"
    folder_name=None


In [8]:
#calculation
from verification import calc_eta
if standard_directory:
    prepath=None
    folder_name='CFBData'
df_res= calc_eta(idx_s, start_id, end_id, step, extract_from ='results', folder_name=folder_name, with_eta_stresses=True, verbalise=False,prepath=prepath)

#print
df_res

There were 0 Structured that resulted in an error during analysis. (Error IDs: [])


,eta_min_s,x_s,y_s,z_s,Location_s,GP_count_s,eta_min_c,x_c,y_c,z_c,...,y_c_shear_reduced,z_c_shear_reduced,Location_c_shear_reduced,element_count_shear_reduced,eta_min_c_stresses,x_c_stresses,y_c_stresses,z_c_stresses,Location_c_stresses,ID
0,1.726060,2060.129150,2149.503906,-194.444443,eta_1,0,2.762616,2060.129150,2450.496094,-194.444443,...,0.0,-479.35,bot,12,1.688713,2060.129150,2450.496094,-194.444443,eta_y_top_stresses,0
0,1.774950,2701.649170,2149.503906,-194.444443,eta_1,0,2.784141,3072.030762,2149.503906,-194.444443,...,0.0,-479.35,bot,13,1.700059,3072.030762,2450.496094,-194.444443,eta_y_top_stresses,1
0,1.694654,3343.169189,2149.503906,-194.444443,eta_1,0,2.625410,3343.169189,2450.496094,-194.444443,...,0.0,-479.35,bot,13,1.624928,3343.169189,2450.496094,-194.444443,eta_y_top_stresses,2
0,1.870838,3713.550781,2450.496094,-194.444443,eta_1,0,2.891899,3713.550781,2149.503906,-194.444443,...,0.0,-479.35,bot,10,1.751701,3713.550781,2450.496094,-194.444443,eta_y_top_stresses,3
0,1.856056,3984.689209,2149.503906,-194.444443,eta_1,0,2.865653,3984.689209,2450.496094,-194.444443,...,0.0,-479.35,bot,11,1.737544,3984.689209,2149.503906,-194.444443,eta_y_top_stresses,4
0,1.883005,4355.070801,2149.503906,-194.444443,eta_1,0,2.907564,4355.070801,2450.496094,-194.444443,...,0.0,-479.35,bot,10,1.755790,4355.070801,2450.496094,-194.444443,eta_y_top_stresses,5
0,1.905306,4626.208984,2149.503906,-194.444443,eta_1,0,2.938690,4626.208984,2149.503906,-194.444443,...,0.0,-479.35,bot,9,1.770641,4626.208984,2450.496094,-194.444443,eta_y_top_stresses,6
0,1.937475,4626.208984,2149.503906,-194.444443,eta_1,0,2.995196,4626.208984,2149.503906,-194.444443,...,0.0,-479.35,bot,9,1.797700,4626.208984,2450.496094,-194.444443,eta_y_top_stresses,7
0,1.910182,4996.590820,2450.496094,-194.444443,eta_1,0,2.943333,4996.590820,2149.503906,-194.444443,...,0.0,-479.35,bot,9,1.773566,4996.590820,2450.496094,-194.444443,eta_y_top_stresses,8
0,1.832151,5267.729004,2149.503906,-194.444443,eta_1,0,2.821203,5267.729004,2149.503906,-194.444443,...,0.0,-479.35,bot,11,1.714598,5267.729004,2450.496094,-194.444443,eta_y_top_stresses,9


In [9]:
df_res[['eta_min_c_stresses','eta_min_c','eta_min_s','eta_min_shear_reduced', 'eta_min_shear']]

,eta_min_c_stresses,eta_min_c,eta_min_s,eta_min_shear_reduced,eta_min_shear
0,1.688713,2.762616,1.726060,0.645153,0.645153
0,1.700059,2.784141,1.774950,0.579676,0.579676
0,1.624928,2.625410,1.694654,0.542821,0.542821
0,1.751701,2.891899,1.870838,0.592530,0.592530
0,1.737544,2.865653,1.856056,0.600713,0.600713
0,1.755790,2.907564,1.883005,0.605428,0.605428
0,1.770641,2.938690,1.905306,0.607816,0.607816
0,1.797700,2.995196,1.937475,0.650090,0.650090
0,1.773566,2.943333,1.910182,0.613325,0.613325
0,1.714598,2.821203,1.832151,0.575149,0.575149


In [10]:
#saving calclulated etas in csv-file
if standard_directory:
    current_directory = os.getcwd()
    folder_path = os.path.join(current_directory, folder_name)
if not standard_directory:
    folder_path=prepath


filepath=folder_path+'\\{}_Batch\\{}_CFB_evaluations.csv'.format(idx_s,idx_s)

df_res.to_csv(filepath,index=True)

In [11]:
#merge x and y df and save them too

y_file_name='\\{}_Batch\\{}_CFB_evaluations.csv'.format(idx_s,idx_s)
y_file_path = folder_path+ y_file_name
df_y=pd.read_csv(y_file_path).drop(columns=['Unnamed: 0'])

x_file_name='\\{}_Batch\\{}_CFBSamples.csv'.format(idx_s,idx_s)
x_file_path = folder_path+ x_file_name
df_x=pd.read_csv(x_file_path).rename(columns={'Unnamed: 0':'ID'})

df_xy = pd.merge(df_x, df_y, on='ID', how='inner')

filepath=folder_path+'\\{}_Batch\\{}_CFB_xy.csv'.format(idx_s,idx_s)
df_xy.to_csv(filepath,index=True)

df_xy

,ID,beta,L,b1,t_p,t_w,h_w,alpha_l,h_v,l_v,...,x_c_shear_reduced,y_c_shear_reduced,z_c_shear_reduced,Location_c_shear_reduced,element_count_shear_reduced,eta_min_c_stresses,x_c_stresses,y_c_stresses,z_c_stresses,Location_c_stresses
0,0,-50.000000,5000,9720,250,400,2780,90,0,0,...,2886.84,0.0,-479.35,bot,12,1.688713,2060.129150,2450.496094,-194.444443,eta_y_top_stresses
1,1,-42.857143,5000,9720,250,400,2780,90,0,0,...,3528.36,0.0,-479.35,bot,13,1.700059,3072.030762,2450.496094,-194.444443,eta_y_top_stresses
2,2,-35.714286,5000,9720,250,400,2780,90,0,0,...,3528.36,0.0,-479.35,bot,13,1.624928,3343.169189,2450.496094,-194.444443,eta_y_top_stresses
3,3,-28.571429,5000,9720,250,400,2780,90,0,0,...,4169.88,0.0,-479.35,bot,10,1.751701,3713.550781,2450.496094,-194.444443,eta_y_top_stresses
4,4,-21.428571,5000,9720,250,400,2780,90,0,0,...,4169.88,0.0,-479.35,bot,11,1.737544,3984.689209,2149.503906,-194.444443,eta_y_top_stresses
5,5,-14.285714,5000,9720,250,400,2780,90,0,0,...,4169.88,0.0,-479.35,bot,10,1.755790,4355.070801,2450.496094,-194.444443,eta_y_top_stresses
6,6,-7.142857,5000,9720,250,400,2780,90,0,0,...,4811.40,0.0,-479.35,bot,9,1.770641,4626.208984,2450.496094,-194.444443,eta_y_top_stresses
7,7,0.000000,5000,9720,250,400,2780,90,0,0,...,4811.40,0.0,-479.35,bot,9,1.797700,4626.208984,2450.496094,-194.444443,eta_y_top_stresses
8,8,7.142857,5000,9720,250,400,2780,90,0,0,...,4811.40,0.0,-479.35,bot,9,1.773566,4996.590820,2450.496094,-194.444443,eta_y_top_stresses
9,9,14.285714,5000,9720,250,400,2780,90,0,0,...,5452.92,0.0,-479.35,bot,11,1.714598,5267.729004,2450.496094,-194.444443,eta_y_top_stresses


## Exploring an individual results dict


In [10]:
# Input HERE
idx_s=1
id = 0
step='step_4'

In [ ]:
#load results dict
from verification import get_results
results=get_results(idx_s, id, step=None)

In [ ]:
results

In [ ]:
#print available keys of the dict
results[step]['GP'].keys()

In [ ]:
#specific discplacement
results[step]['nodal']['uz']['20']

In [ ]:
#specific stresses
results[step]['GP']['sig_sr_1L_x']['99']

In [ ]:
#calculating specific etas
from verification import *
concrete_bending_verification_stresses(structure=None, results = results, step=step, return_type='dict', verbalise=True)

## Check Valdiation functions

In [ ]:
# Input HERE
idx_s=1
id = 0
step='step_4'

In [ ]:
#load results dict
from verification import get_results
results=get_results(idx_s, id, step)

#### Check reinforcement steel bending verification

In [ ]:
#calculate maximum stresses
max_sig=[max(results['GP']['sig_sr_1L_x'].values()),
         max(results['GP']['sig_sr_1L_y'].values()),
         max(results['GP']['sig_sr_2L_x'].values()),
         max(results['GP']['sig_sr_2L_y'].values()),
         max(results['GP']['sig_sr_3L_x'].values()),
         max(results['GP']['sig_sr_3L_y'].values()),
         max(results['GP']['sig_sr_4L_x'].values()),
         max(results['GP']['sig_sr_4L_y'].values()),
]
print(max_sig)

In [ ]:
#!!compare with calculated eta_min_s
720/max(max_sig)

#### Check concrete bending verification (strains)

In [ ]:
#calculate minimum strains
min_eps=[min(results['GP']['eps_1_bot'].values()),
         min(results['GP']['eps_3_bot'].values()),
         min(results['GP']['eps_1_top'].values()),
         min(results['GP']['eps_3_top'].values())        
]
print(min_eps)

In [ ]:
#!!compare with calculated eta_min_c
-0.002/min(min_eps)

#### Check concrete bending verification (stresses)

In [ ]:
#calculate maximum negative stresses (--> minimum stresses)
# min_eps=[min(results['GP']['eps_1_bot'].values()),
#          min(results['GP']['eps_3_bot'].values()),
#          min(results['GP']['eps_1_top'].values()),
#          min(results['GP']['eps_3_top'].values())        
# ]
# print(min_eps)

In [ ]:
#!!compare with calculated eta_min_c_stresses
# fcc_eff/sig_c
# -0.002/min(min_eps)

In [ ]:
# results['GP']['fcc_eff_top'] 

In [ ]:
# results['GP']['sig_x_top']

## Plotting Runtimes

In [ ]:
#saving runtimes
n_elements1=[2450.0, 1601.0, 1142.0, 947.0,  947.0, 947.0, 947.0, 947.0, 947.0, 947.0]
analysis_time1=[1052.815,1011.080,649.90,633.543,632.563,637.594,641.200, 650.680, 650.564, 662.114] #seconds
analysis_time1 = [element / 60 for element in analysis_time1] #minutes

n_elements2= [362.0, 257.0,  206.0, 128.0, 128.0, 128.0,  128.0, 128.0,  128.0, 128.]
analysis_time2=[83.119,92.788 ,102.771,84.002,88.768 ,82.442,85.991,87.115, 84.954, 84.954]
analysis_time2 = [element / 60 for element in analysis_time2] #minutes

n_elements3=[2756.0, 1835.0, 1232.0, 1028.0, 1028.0, 1028.0, 1028.0, 1028.0, 1028.0, 1028.0]
analysis_time3=[1107.244,911.303, 570.685, 602.179, 594.138, 591.931, 593.499, 606.966, 603.610, 600.916]
analysis_time3 = [element / 60 for element in analysis_time3] #minutes


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(n_elements1, analysis_time1,label='L5800_parameterstudie (ms=1t)', marker='o', color='darkblue')
ax.plot(n_elements2, analysis_time2,label='L5800_parameterstudie (ms=3t)', marker='X', color='blue')
ax.plot(n_elements3, analysis_time3,label='L7000_parameterstudie (ms=1t)', marker='.', color='lightblue')


ax.set(xlabel='number of elements', ylabel='analysis time [minutes]',
       title='Runtime comparison')

# ax.set_ylim(bottom=0,top=20)

ax.grid()
ax.legend()

#fig.savefig("test.png")
plt.show()